# WeaviateStore

[Weaviate](https://weaviate.io/) is an open source vector database that stores both objects and vectors, allowing for combining vector search with structured filtering. LangChain connects to Weaviate via the weaviate-ts-client package, the official Typescript client for Weaviate.

This guide provides a quick overview for getting started with Weaviate [vector stores](/docs/concepts/#vectorstores). For detailed documentation of all `WeaviateStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/vectorstores/weaviate/) |  Package latest |
| :--- | :--- | :---: | :---: |
| [`WeaviateStore`](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html) | [`@langchain/weaviate`](https://npmjs.com/@langchain/weaviate) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/weaviate?style=flat-square&label=%20&) |

## Setup

To use Weaviate vector stores, you'll need to set up a Weaviate instance and install the `@langchain/weaviate` integration package. You should also install the `weaviate-ts-client` package to initialize a client to connect to your instance with, and the `uuid` package if you want to assign indexed documents ids.

This guide will also use [OpenAI embeddings](/docs/integrations/text_embedding/openai), which require you to install the `@langchain/openai` integration package. You can also use [other supported embeddings models](/docs/integrations/text_embedding) if you wish.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/weaviate @langchain/core weaviate-ts-client uuid @langchain/openai
</Npm2Yarn>
```

You'll need to run Weaviate either locally or on a server. See [the Weaviate documentation](https://weaviate.io/developers/weaviate/installation) for more information.

### Credentials

Once you've set up your instance, set the following environment variables:

```typescript
// http or https
process.env.WEAVIATE_SCHEME = "";
// If running locally, include port e.g. "localhost:8080"
process.env.WEAVIATE_HOST = "YOUR_HOSTNAME";
// Optional, for cloud deployments
process.env.WEAVIATE_API_KEY = "YOUR_API_KEY";
```

If you are using OpenAI embeddings for this guide, you'll need to set your OpenAI key as well:

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```typescript
// process.env.LANGCHAIN_TRACING_V2="true"
// process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

In [1]:
import { WeaviateStore } from "@langchain/weaviate";
import { OpenAIEmbeddings } from "@langchain/openai";

import weaviate from "weaviate-ts-client";
// import { ApiKey } from "weaviate-ts-client"

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

// The Weaviate SDK has an issue with types
const weaviateClient = (weaviate as any).client({
  scheme: process.env.WEAVIATE_SCHEME ?? "http",
  host: process.env.WEAVIATE_HOST ?? "localhost",
  // If necessary
  // apiKey: new ApiKey(process.env.WEAVIATE_API_KEY ?? "default"),
});

const vectorStore = new WeaviateStore(embeddings, {
  client: weaviateClient,
  // Must start with a capital letter
  indexName: "Langchainjs_test",
  // Default value
  textKey: "text",
  // Any keys you intend to set as metadata
  metadataKeys: ["source"],
});

## Manage vector store

### Add items to vector store

**Note:** If you want to associate ids with your indexed documents, they must be UUIDs.

In [2]:
import type { Document } from "@langchain/core/documents";
import { v4 as uuidv4 } from "uuid";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];
const uuids = [uuidv4(), uuidv4(), uuidv4(), uuidv4()];

await vectorStore.addDocuments(documents, { ids: uuids });

[
  '610f9b92-9bee-473f-a4db-8f2ca6e3442d',
  '995160fa-441e-41a0-b476-cf3785518a0d',
  '0cdbe6d4-0df8-4f99-9b67-184009fee9a2',
  '18a8211c-0649-467b-a7c5-50ebb4b9ca9d'
]


### Delete items from vector store

You can delete by id as by passing a `filter` param:

In [3]:
await vectorStore.delete({ ids: [uuids[3]] });

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [14]:
const filter = {
  where: {
    operator: "Equal" as const,
    path: ["source"],
    valueText: "https://example.com",
  }
};

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


See [this page](https://weaviate.io/developers/weaviate/api/graphql/filters) for more on Weaviat filter syntax.

If you want to execute a similarity search and receive the corresponding scores you can run:

In [15]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.835] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.852] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### Query by turning into retriever

You can also transform the vector store into a [retriever](/docs/concepts/retrievers) for easier usage in your chains. 

In [16]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials: working with external knowledge](/docs/tutorials/#working-with-external-knowledge).
- [How-to: Question and answer with RAG](/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](/docs/concepts/retrieval)

## API reference

For detailed documentation of all `WeaviateStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html).